# 03 - Expériences quantiques (QSVM)

Ce notebook illustre l'utilisation du module quantique et du QuantumSVM,
ainsi que la comparaison avec un SVM classique sur un sous-ensemble des features.


In [ ]:
from pathlib import Path
import h5py
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

from src.models import QuantumSVM
from src.models.quantum_svm import QuantumSVMConfig
from src.quantum_module import compute_quantum_gram_matrix

features_path = Path('results/features/extracted_features.h5')
features_path


In [ ]:
if features_path.exists():
    with h5py.File(features_path, 'r') as f:
        X = np.array(f['X'])
        y = np.array(f['y'])
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    # Sous-échantillonnage pour QSVM
    max_q_train = min(30, X_train.shape[0])
    X_train_q = X_train[:max_q_train]
    y_train_q = y_train[:max_q_train]
    X_train_q.shape, X_test.shape


In [ ]:
# Entraînement SVM classique pour comparaison
if features_path.exists():
    svm = SVC(kernel='rbf', probability=True, random_state=42)
    svm.fit(X_train, y_train)
    svm_acc = svm.score(X_test, y_test)
    print('Accuracy SVM RBF:', svm_acc)


In [ ]:
# Entraînement QuantumSVM sur un sous-ensemble
if features_path.exists():
    cfg = QuantumSVMConfig(n_qubits=8, shots=512, backend_name='aer_simulator')
    qsvm = QuantumSVM(config=cfg)
    qsvm.fit(X_train_q, y_train_q)
    q_acc = (qsvm.predict(X_test) == y_test).mean()
    print('Accuracy QSVM:', q_acc)
